In [11]:
!pip install transformers

     |████████████████████████████████| 1.4MB 5.6MB/s 
     |████████████████████████████████| 2.9MB 44.7MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6a4e67232e593e165b812cb9192cef3fdffdb593c8811b2debe7e45ab53581b8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [37]:
import os
import random
import collections
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch
import pprint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [38]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/text2image'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
categories =  [
'Основные блюда',
'Курица',
'Рыба',              
'Мороженое',
'Пирожное',
'Пудинг',
'Лапша',
'Пюре',
'Шашлык',
'Булочки',
'Стейки',
'Сырники',
'Яичница',
'Конфеты',
'Плов',
'Соусы и маринады',
'Паштеты',
'Роллы',
'Тосты',
'Суфле',
'Соленья и консервация',
'Лазанья',
'Гамбургер',
'Жаркое',
'Салаты',
'Супы',
'Пироги',
'Торты',
'Печенье',
'Сэндвичи',
'Запеканка',
'Кексы',
'Блины',
'Хлеб',
'Котлеты',
'Каши',
'Напитки',
'Омлет',
'Оладьи']

In [40]:
category_mapping = {
    'Пирожки':'Пироги',
    'Чизбургер':'Гамбургер',        
    'Пирожное': 'Торты',
}

In [41]:
def map_title(title):
  if title.startswith( 'Курица' ):
    return 'Курица'
  if title.startswith( 'Форель' ):
    return 'Рыба'
  if title.startswith( 'Рыба' ):
    return 'Рыба'
  if title.startswith( 'Ватрушка' ):
    return 'Пироги'
  if title.startswith( 'Жюльен' ):
    return 'Жаркое'
  if title.startswith( 'Индейка' ):
    return 'Курица'
  if title.startswith( 'Спагетти' ):
    return 'Лапша'     
  if title.startswith( 'Куриные крылышки' ):
    return 'Курица'  
  if title.startswith( 'Куриные грудки' ):
    return 'Курица'  
  if title.startswith( 'Пицца' ):
    return 'Пироги'
  if title.startswith( 'Фунчоза' ):
    return 'Лапша'
  if title.startswith( 'Запеченый лосось' ):
    return 'Рыба'
  if title.startswith( 'Профитроли' ):
    return 'Булочки'  
    
  return None

In [42]:
def have_single_strategy(tags, categories):
    actual_categories = list(set(tags) & set(categories))
    if len(actual_categories) == 1:
      return actual_categories[0]
    return None
def first_from_tags_strategy(tags, categories):
    categories = set(categories)
    for tag in tags:
      if tag in categories:
        return tag
    return None
def first_from_categories_strategy(tags, categories):
    tags = set(tags)
    for category in categories:
      if category in tags:
        return category
    return None

In [43]:
print(have_single_strategy(['Лазанья'], ['Суфле','Лазанья']))
print(have_single_strategy(['Лазанья', 'Суфле'], ['Суфле','Лазанья']))
print(first_from_tags_strategy(['Лазанья','Суфле'], ['Суфле','Лазанья']))
print(first_from_categories_strategy(['Лазанья','Суфле'], ['Суфле','Лазанья']))

Лазанья
None
Лазанья
Суфле


In [44]:
if not os.path.exists('GDriveDL.py'):
    !wget https://raw.githubusercontent.com/matthuisman/gdrivedl/master/gdrivedl.py -O GDriveDL.py

os.makedirs('data', exist_ok=True)
if not os.path.exists('data/eda_ru.zip'): 
    !python GDriveDL.py https://drive.google.com/file/d/1CNIbj8_OuxQD74zt6JU4BUI8ngctutl9/view?usp=sharing data
    !unzip -q data/eda_ru.zip -d data

In [45]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
rubert = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=len(categories))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
def get_recipe_text(row):
    title_text = row['title'] 
    ingredients_text = row['ingredients'].replace('|', ' ')
    steps_text = row['steps'].replace('|', ' ')
    return f"{title_text} {ingredients_text} {steps_text}"

In [47]:
def replace_tags(tags):
    new_tags = []
    for tag in tags:
        if tag in category_mapping:
            new_tags.append(category_mapping[tag])
        else:
            new_tags.append(tag)
    return list(set(new_tags))

In [48]:
train_field = 'full text'
#train_field = 'title'

In [49]:
data = pd.read_csv('./data/eda_ru_filtered_with_tags.csv', sep=',')
data["tags"]= data["tags"].str.split(pat="|")

data["tags"]= data["tags"].apply(lambda x: replace_tags(x))
data["full text"] = data.apply(lambda x: get_recipe_text(x), axis=1)

#tag_selection_stratagy = have_single_strategy
#tag_selection_stratagy = first_from_tags_strategy
tag_selection_stratagy = first_from_categories_strategy

def select_tag(row):
  tag = tag_selection_stratagy(row['tags'], categories)
  if tag is None:
    tag = map_title(row['title'])
  return tag

data["main tag"] = data.apply(lambda x: select_tag(x), axis=1)
data_without_tag = data[data['main tag'].isna()]
data_with_tag = data[data['main tag'].notna()]

train, test = train_test_split(data_with_tag, test_size=0.1, random_state=42, shuffle=True)

train_token = tokenizer(train[train_field].astype(str).values.tolist(), truncation=True, padding=True, max_length=512)
test_token = tokenizer(test[train_field].astype(str).values.tolist(), truncation=True, padding=True, max_length=512)

def get_labels(data, labels):
  return data["main tag"].apply(lambda x: labels.index(x)).values.tolist()

train_label = get_labels(train, categories)
test_label = get_labels(test, categories)

In [50]:
len(train_label)

19618

In [51]:
class FoodDataSet(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = FoodDataSet(train_token, train_label)
test_dataset = FoodDataSet(test_token, test_label)

In [52]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=4,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=32,  
    warmup_steps=500,               
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=100,
)

In [53]:
trainer = Trainer(
    model=rubert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [54]:
torch.cuda.empty_cache()
trainer.train()

Step,Training Loss
100,3.201841
200,2.292773
300,1.593479
400,1.200583
500,0.988645
600,0.957663
700,0.794469
800,0.781921
900,0.705364
1000,0.610502


TrainOutput(global_step=9812, training_loss=0.44010945383111555)

In [55]:
rubert.save_pretrained('./drive/My Drive/text2image/class-model')
rubert = BertForSequenceClassification.from_pretrained('./drive/My Drive/text2image/class-model')

In [56]:
device = torch.device("cuda:0")
rubert = rubert.to(device)

In [57]:
def predict(text):
    inputs = tokenizer([text], truncation=True, padding=True, max_length=512, return_tensors="pt").to(rubert.device)
    outputs = rubert(**inputs,labels=None)
    logits = outputs.logits.cpu().detach().numpy()
    categoryIndex = np.argmax(logits)
    return categories[categoryIndex]

In [58]:
predict('Яйца мука молоко взбить пожарить')

'Яичница'

In [59]:
from tqdm import tqdm

res = []

for i, row in  tqdm(test.iterrows(), position=0, leave=True):
    res.append(predict(row[train_field]))

2180it [00:39, 55.07it/s]


In [61]:
len(res)
from sklearn.metrics import accuracy_score

r = [categories.index(x) for x in res]
accuracy_score(test_label, r)

0.9073394495412844

In [26]:
import collections
counter = collections.Counter()

res = {}
for i, row in  tqdm(data.iterrows(), position=0, leave=True):
    p = predict(row[train_field])
    res[row['id']] = p
    counter.update([p])

30028it [10:47, 46.37it/s]


In [31]:
respd = pd.DataFrame(res.items())
respd.columns = ['id', 'category']
respd.to_csv('categories.csv',index=False)

RangeIndex(start=0, stop=2, step=1)